In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.stat import Correlation

#### Requêtes SQL 

In [2]:
print(pyspark.__version__)

3.1.2


In [3]:
conf = pyspark.SparkConf().set("spark.jars.packages", 
                                "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1").setMaster("local").setAppName("My App").setAll([("spark.driver.memory", "40g"), ("spark;executor.memory", "50g")])

In [4]:
sc = SparkContext(conf=conf) #Connected to pyspark

In [5]:
sqlC = SQLContext(sc)

In [6]:
mongo_ip = "mongodb://localhost:27017/restaurantsdb."

In [7]:
restaurants = sqlC.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip + "restaurants").load()

In [8]:
restaurants.createOrReplaceTempView("restaurants")

In [9]:
restaurants = sqlC.sql("SELECT * FROM restaurants")

In [10]:
restaurants.show()

+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------+
|                 _id|             address|      borough|             cuisine|              grades|                name|restaurant_id|
+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------+
|{61f15a93e0cb0c13...|{1007, [-73.85607...|        Bronx|              Bakery|[{2014-03-03 01:0...|Morris Park Bake ...|     30075445|
|{61f15a93e0cb0c13...|{469, [-73.961704...|     Brooklyn|          Hamburgers|[{2014-12-30 01:0...|             Wendy'S|     30112340|
|{61f15a93e0cb0c13...|{351, [-73.985135...|    Manhattan|               Irish|[{2014-09-06 02:0...|Dj Reynolds Pub A...|     30191841|
|{61f15a93e0cb0c13...|{2780, [-73.98241...|     Brooklyn|           American |[{2014-06-10 02:0...|     Riviera Caterer|     40356018|
|{61f15a93e0cb0c13...|{97-22, [-73.8601...|       Queen

In [11]:
# Nombre de données dans la table
restaurants = sqlC.sql("SELECT COUNT(*) FROM restaurants")
restaurants.show()

+--------+
|count(1)|
+--------+
|    3772|
+--------+



In [12]:
# Nombre de restaurants par code postal
restaurants = sqlC.sql("SELECT address.zipcode, COUNT(restaurant_id) FROM restaurants GROUP BY address.zipcode")
restaurants.show()

+-------+--------------------+
|zipcode|count(restaurant_id)|
+-------+--------------------+
|  11205|                  10|
|  11236|                  11|
|  10309|                  13|
|  11106|                  27|
|  11218|                  16|
|  10452|                  11|
|  11428|                   5|
|  11237|                   9|
|  11379|                  11|
|  11364|                  10|
|  11249|                   8|
|  10012|                  94|
|  11001|                   2|
|  11385|                  34|
|  11238|                  12|
|  10039|                   4|
|  11427|                   5|
|  11367|                  10|
|  10010|                  27|
|  10038|                  18|
+-------+--------------------+
only showing top 20 rows



In [13]:
# Nombre de restaurants par type de cuisine
restaurants = sqlC.sql("SELECT cuisine, COUNT(restaurant_id) FROM restaurants GROUP BY cuisine")
restaurants.show()

+----------------+--------------------+
|         cuisine|count(restaurant_id)|
+----------------+--------------------+
|Pancakes/Waffles|                   7|
|Chinese/Japanese|                   1|
|         Mexican|                  73|
|   Jewish/Kosher|                  60|
|          Bakery|                 127|
|         Turkish|                  11|
|        Armenian|                   1|
|         Hotdogs|                   4|
|       Ethiopian|                   3|
|            Thai|                  14|
|          Indian|                  43|
|         Chinese|                 115|
|      Indonesian|                   2|
|       Soul Food|                   6|
|     Continental|                   8|
|           Steak|                  21|
|         African|                   4|
|CafÃ©/Coffee/Tea|                   1|
|          Donuts|                  43|
|           Tapas|                   4|
+----------------+--------------------+
only showing top 20 rows



In [14]:
# Le restaurant le mieux noté
from pyspark.sql.functions import *

restaurants = sqlC.sql("select cuisine ,aggregate(grades.score,0,(x,y) -> x + y) as sum from restaurants where cuisine = 'American '")
restaurants.show()

+---------+---+
|  cuisine|sum|
+---------+---+
|American | 36|
|American | 68|
|American | 48|
|American | 18|
|American | 14|
|American | 13|
|American | 61|
|American | 37|
|American | 67|
|American | 31|
|American | 66|
|American | 39|
|American | 47|
|American | 39|
|American | 50|
|American | 13|
|American | 63|
|American | 60|
|American | 81|
|American | 62|
+---------+---+
only showing top 20 rows



### Preprocessing

In [15]:
restaurants_df = sqlC.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip + "restau_df").load()

In [16]:
restaurants_df.createOrReplaceTempView("restau_df")

In [17]:
restaurants_df.show()

+--------------------+--------------------+-----+-----------------+------------------+-------------+-------------+-------+
|                 _id|             cuisine|grade|        lattitude|         longitude|nb_infraction|     quartier|zipcode|
+--------------------+--------------------+-----+-----------------+------------------+-------------+-------------+-------+
|{61f15a93e0cb0c13...|              Bakery|    A|        40.848447|        -73.856077|            2|        Bronx|  10462|
|{61f15a93e0cb0c13...|          Hamburgers|    A|        40.662942|        -73.961704|            8|     Brooklyn|  11225|
|{61f15a93e0cb0c13...|               Irish|    A|       40.7676919|-73.98513559999999|            2|    Manhattan|  10019|
|{61f15a93e0cb0c13...|           American |    A|        40.579505|-73.98241999999999|            5|     Brooklyn|  11224|
|{61f15a93e0cb0c13...|       Jewish/Kosher|    Z|       40.7311739|       -73.8601152|           20|       Queens|  11374|
|{61f15a93e0cb0c

In [18]:
restaurants_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- cuisine: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- lattitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- nb_infraction: integer (nullable = true)
 |-- quartier: string (nullable = true)
 |-- zipcode: string (nullable = true)



In [19]:
restaurants_df.describe().show()

+-------+--------------------+-----+------------------+------------------+------------------+-------------+-----------------+
|summary|             cuisine|grade|         lattitude|         longitude|     nb_infraction|     quartier|          zipcode|
+-------+--------------------+-----+------------------+------------------+------------------+-------------+-----------------+
|  count|                3772| 3772|              3772|              3772|              3772|         3772|             3772|
|   mean|                null| null| 40.67625120771491|-73.96846774331894|10.701219512195122|         null|10545.26802757158|
| stddev|                null| null|1.6256337301241979| 4.953221264543527|  5.98845956338147|         null| 588.991762883543|
|    min|              Afghan|    A|       -28.0168595|      -119.6368672|                -1|        Bronx|            10001|
|    max|Vietnamese/Cambod...|    Z|        51.6514664|       153.1628795|                89|Staten Island|           

### Preprocessing

In [20]:
import pymongo
import pandas as pd
from pymongo import MongoClient

#MongoDB collection to Pandas dataframe
c = MongoClient()
db = c.restaurantsdb
input_data = db.restau_df
df = pd.DataFrame(list(input_data.find()))
df

,_id,longitude,lattitude,zipcode,cuisine,quartier,nb_infraction,grade
0,61f15a93e0cb0c134b208f69,-73.856077,40.848447,10462,Bakery,Bronx,2,A
1,61f15a93e0cb0c134b208f6a,-73.961704,40.662942,11225,Hamburgers,Brooklyn,8,A
2,61f15a93e0cb0c134b208f6b,-73.985136,40.767692,10019,Irish,Manhattan,2,A
3,61f15a93e0cb0c134b208f6c,-73.982420,40.579505,11224,American,Brooklyn,5,A
4,61f15a93e0cb0c134b208f6d,-73.860115,40.731174,11374,Jewish/Kosher,Queens,20,Z
...,...,...,...,...,...,...,...,...
3767,61f15a94e0cb0c134b209e20,-73.986544,40.733770,10003,Café/Coffee/Tea,Manhattan,5,A
3768,61f15a94e0cb0c134b209e21,-73.946224,40.656571,11203,Café/Coffee/Tea,Brooklyn,3,A
3769,61f15a94e0cb0c134b209e22,-73.983944,40.756111,10036,Japanese,Manhattan,7,A
3770,61f15a94e0cb0c134b209e23,-73.986652,40.753641,10018,Sandwiches/Salads/Mixed Buffet,Manhattan,5,A


In [21]:
df['cuisine'].value_counts()

American            1255
Italian              325
Pizza                270
Café/Coffee/Tea      180
Hamburgers           159
                    ... 
CafÃ©/Coffee/Tea       1
Moroccan               1
Armenian               1
Chinese/Japanese       1
Salads                 1
Name: cuisine, Length: 70, dtype: int64

In [22]:
df['quartier'].value_counts()

Manhattan        1883
Queens            738
Brooklyn          684
Bronx             309
Staten Island     158
Name: quartier, dtype: int64

In [23]:
df['grade'].value_counts()

A                 3312
B                  253
Z                  181
C                   25
Not Yet Graded       1
Name: grade, dtype: int64

##### Label Encoding

In [24]:
df["zipcode"] = df["zipcode"].astype(str)
one_hot = pd.get_dummies(df['zipcode'])
df = df.drop('zipcode',axis = 1)
df = df.join(one_hot)
one_hot = pd.get_dummies(df['quartier'])
df = df.drop('quartier',axis = 1)
df = df.join(one_hot)
one_hot = pd.get_dummies(df['cuisine'])
df = df.drop('cuisine',axis = 1)
df = df.join(one_hot)

df.head()

,_id,longitude,lattitude,nb_infraction,grade,10001,10002,10003,10004,10005,...,Soul Food,Soups & Sandwiches,Spanish,Steak,Tapas,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese/Cambodian/Malaysia
0,61f15a93e0cb0c134b208f69,-73.856077,40.848447,2,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,61f15a93e0cb0c134b208f6a,-73.961704,40.662942,8,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,61f15a93e0cb0c134b208f6b,-73.985136,40.767692,2,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,61f15a93e0cb0c134b208f6c,-73.982420,40.579505,5,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,61f15a93e0cb0c134b208f6d,-73.860115,40.731174,20,Z,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
del df['_id']
del df['grade']
df.head()

,longitude,lattitude,nb_infraction,10001,10002,10003,10004,10005,10006,10007,...,Soul Food,Soups & Sandwiches,Spanish,Steak,Tapas,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese/Cambodian/Malaysia
0,-73.856077,40.848447,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-73.961704,40.662942,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-73.985136,40.767692,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-73.982420,40.579505,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-73.860115,40.731174,20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_y = df['nb_infraction']
del df['nb_infraction']
df['nb_infraction'] = df_y
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

df.head()

,longitude,lattitude,10001,10002,10003,10004,10005,10006,10007,10009,...,Soups & Sandwiches,Spanish,Steak,Tapas,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese/Cambodian/Malaysia,nb_infraction
0,-73.856077,40.848447,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,-73.961704,40.662942,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
2,-73.985136,40.767692,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,-73.982420,40.579505,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,-73.860115,40.731174,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20


In [27]:
X.head()

,longitude,lattitude,10001,10002,10003,10004,10005,10006,10007,10009,...,Soul Food,Soups & Sandwiches,Spanish,Steak,Tapas,Tex-Mex,Thai,Turkish,Vegetarian,Vietnamese/Cambodian/Malaysia
0,-73.856077,40.848447,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-73.961704,40.662942,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-73.985136,40.767692,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-73.982420,40.579505,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-73.860115,40.731174,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
y.head()

0     2
1     8
2     2
3     5
4    20
Name: nb_infraction, dtype: int64

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

In [31]:

# evaluate random forest ensemble for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
model.fit(X_train, y_train)
model.score(X_test, y_test)

-0.22738734046551867

- Quartier et Cuisine en catégorie
- Extraire grade et score
- Extraire longitude et latitude (variable adress)
- Split train set et test set
- Regression : transformer output en classe et comparer avec le modele de classification
- Choisir metrics de regression et classification
- Arbre de décision, Random Forest, Xgboost

Remarques : voir le nombre de controles par restaurant et le grade donné à chaque fois. 